In [34]:
#uploading different packages- to remove the one we do not need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Imputer
import os
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics 
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import LeaveOneOut,ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cross_validation import StratifiedKFold
from random import seed
seed(123)
rcParams['figure.figsize'] = 12, 4


In [2]:
"""
#set path to imprt and save files from and in
with open('PATHS.json') as json_data:
    paths = json.load(json_data)

#upload data
train = pd.read_csv(paths["TRAIN_PATH"],index_col='Unnamed: 0')
test = pd.read_csv(paths["TEST_PATH"],index_col='Unnamed: 0')
"""

In [35]:
#set path to imprt and save files from and in
path = 'C:/Users/Yonathan/Desktop/Rafael'

#upload data
train = pd.read_csv(os.path.join(path,r'train.csv'),index_col='Unnamed: 0')
test = pd.read_csv(os.path.join(path,r'test.csv'),index_col='Unnamed: 0')

In [36]:
#remove labels names from data
train=train.drop('targetName', 1)
#remove unnecessary time cells from data
col_names = list(train)
for name in col_names:
    if name[:4] == "Time":
        train=train.drop(name, 1)
        
col_names = list(test)
for name in col_names:
    if name[:4] == "Time":
        test=test.drop(name, 1)
#train.head()

In [165]:
# build a function vec_size which measures vector magnitude
def vec_size(x,y,z):
    return (np.sqrt(z**2+x**2+y**2))
#create a df vel_mag with the magnitude of the velocity and val_mean which average the velocity of the samples(row)   
def vel(data):    
    vel_magn=pd.DataFrame()
    for i in range(30):
        x = data["velX_"+str(i)]
        y = data["velY_"+str(i)]
        z = data["velZ_"+str(i)]
    #velocity magnitude matrix    
        vel_magn["vel_magnitude_"+str(i)] =vec_size(x,y,z)
    return (vel_magn)

vel_mag_train = vel(train)
vel_mag_test = vel(test)
#mean velocity magnitudevector
vel_mean_train = pd.DataFrame(np.mean(vel_mag_train, axis=1),columns=["vel_mean"])
vel_mean_test= pd.DataFrame(np.mean(vel_mag_test, axis=1),columns=["vel_mean"])


In [166]:
#a function to calculate the acceleration between each step
def acc(data, vel_res):   
    acc_df=pd.DataFrame()
    for i in range(0,29):
        vel1=vel_res["vel_magnitude_"+str(i)]
        vel2=vel_res["vel_magnitude_"+str(i+1)]
        acc_df["accel_"+str(i)]=vel2-vel1
    return (acc_df) 

acc_df_train =acc(train, vel_mag_train)
acc_df_test =acc(test, vel_mag_test)
#mean acc 
acc_mean_train=pd.DataFrame(np.mean(acc_df_train, axis=1),columns=["acc_mean"])
acc_mean_test=pd.DataFrame(np.mean(acc_df_test, axis=1),columns=["acc_mean"])
#print (acc_df)

In [180]:
acc_df_train

,accel_0,accel_1,accel_2,accel_3,accel_4,accel_5,accel_6,accel_7,accel_8,accel_9,accel_10,accel_11,accel_12,accel_13,accel_14,accel_15,accel_16,accel_17,accel_18,accel_19,accel_20,accel_21,accel_22,accel_23,accel_24,accel_25,accel_26,accel_27,accel_28
0,-2.946373,-8.005968,-8.965852,-9.715203,-6.772376,-11.305675,-8.182346,-10.327866,-0.220828,-8.750589,-7.017443,-7.383721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-5.916209,0.095807,-5.081512,-0.708563,3.509726,-6.464790,-4.097747,0.701173,-4.084054,1.294083,-2.385908,-3.492623,7.115608,-9.633581,-1.089797,-1.443118,-0.732297,5.183897,-1.837125,-5.142397,-7.454515,3.303700,-2.569053,0.373441,0.193547,-2.676699,-1.276567,NaN,NaN
2,-5.229192,-4.348946,2.388189,-6.164057,-6.931376,-7.561774,5.394835,-4.058784,-10.447624,4.764084,-4.537255,-8.199319,2.343203,-1.024454,-7.754937,-0.743609,-0.108191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.051633,-7.163126,9.360260,-3.037515,0.748334,-9.466941,6.649618,-7.919813,2.024924,2.315196,-5.443359,0.059418,2.182915,-2.734105,-4.416584,3.465673,3.620161,-2.379339,2.188289,-6.274483,5.035872,-4.214527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,180.886102,191.295367,203.819715,218.748983,235.933404,256.004484,103.820002,-120.098571,-112.907758,-83.393568,-76.367743,-66.437304,-63.353442,-46.038351,-44.586029,-43.319153,-38.206079,-30.618241,-35.957898,-20.954809,-26.990371,-28.484335,-17.344244,-20.509415,-17.573546,-18.167614,-19.622549,-17.426776,NaN
5,-8.647792,3.312839,-1.151627,-2.428661,-7.924288,-7.384844,2.281457,-6.056525,-6.458452,11.189591,-9.461634,-9.730188,4.566542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.799151,-2.447119,0.191862,6.262464,2.966815,2.057734,4.846975,-4.945556,4.711945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-11.683721,-10.043435,-19.078459,-5.725452,-6.847902,-10.173024,-12.995181,-6.740855,-15.924906,-1.488064,-8.369993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-2.331385,6.172207,-0.125278,-0.181309,-3.761175,2.134299,-2.717607,9.448164,-3.428306,5.363869,-2.668462,5.285770,-1.517624,0.332830,6.587756,-3.081207,10.497324,-0.956616,-0.662825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-13.962043,-11.677990,-17.943621,-5.038998,-16.116164,-9.338132,-8.272502,-17.846428,-4.820007,-17.040011,-11.905009,-2.186264,-14.700781,-6.474515,-2.281239,-15.627669,-6.583571,-8.066916,-7.434244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
#angle calculation
def calc_angle(data,pos_index): 
        x_prev = data["posX_"+str(pos_index)] 
        x_curr = data["posX_"+str(pos_index+1)]
        
        y_prev = data["posY_"+str(pos_index)] 
        y_curr = data["posY_"+str(pos_index+1)]
        
        z_prev = data["posZ_"+str(pos_index)] 
        z_curr = data["posZ_"+str(pos_index+1)]
        
        curr_point_vec = [x_curr-x_prev,y_curr-y_prev,z_curr-z_prev]
        curr_point_vec_mag = vec_size(curr_point_vec[0],curr_point_vec[1],curr_point_vec[2])
        curr_point_vec_norm = [curr_point_vec[0]/curr_point_vec_mag,curr_point_vec[1]/curr_point_vec_mag,curr_point_vec[2]/curr_point_vec_mag]
        
        plain_vec =[x_curr-x_prev,y_curr-y_prev,0] 
        plain_vec_mag = vec_size(plain_vec[0],plain_vec[1],0)
        
        plain_vec_norm = [plain_vec[0]/plain_vec_mag,plain_vec[1]/plain_vec_mag,0]
        res = curr_point_vec_norm[0]*plain_vec_norm[0] +curr_point_vec_norm[1]*plain_vec_norm[1] +curr_point_vec_norm[2]* plain_vec_norm[2] 
        angle = np.arccos(res)
        return (angle*180.0/ np.pi)
    
#run it on a whole df
def angle(data, num_of_func = 1):    
    angle_df=pd.DataFrame()
    for i in range(29): #TODO: Why range only until 29?
        #print (train.iloc[:,i])
        if num_of_func==1:
            angle_df["angle_"+str(i)] =calc_angle(data,i)
        else:
            angle_df["angle_"+str(i)] =calc_angle2(data,i)
    return angle_df


In [168]:
angle_df_train=angle(train,1)
angle_df_test=angle(test,1)

angle_df_train2 =angle(train,2)
angle_df_test2 =angle(test,2)
'''
print("angle_df_train: ")
print(angle_df_train.head())
print("angle_df_train2: ")
print(angle_df_train2.head())
'''
#calculate the mean
angle_mean_train = pd.DataFrame(np.mean(angle_df_train, axis=1),columns=["angle_mean"])
angle_mean_test = pd.DataFrame(np.mean(angle_df_test, axis=1),columns=["angle_mean"])
#print (angle_df)

In [169]:
#a df to calculate the change in angles between each step
def angle_che(data,angle_df):
    angle_change_df=pd.DataFrame()
    for i in range(0,28):
        ang1=angle_df["angle_"+str(i)]
        ang2=angle_df["angle_"+str(i+1)]
        angle_change_df["angle_change_"+str(i)]=np.abs(ang2-ang1)
        
    return(angle_change_df)
#print (angle_change_df)


angle_change_df_train = angle_che(train,angle_df_train)
angle_change_df_test = angle_che(test,angle_df_test)

#calculate the mean
angle_change_mean_train = pd.DataFrame(np.mean(angle_change_df_train, axis=1),columns=["angle_change_mean"])
angle_change_mean_test = pd.DataFrame(np.mean(angle_change_df_test, axis=1),columns=["angle_change_mean"])
#print (angle_change_mean)

In [170]:
#count how many time steps each samples has (as non NaN)
def count_time(data):
    res = pd.DataFrame()
    time_vec=[]
    for i in range(0,len(data)):
        sample=data.iloc[i,:]
        time_vec.append((29-sample.isnull().sum()/6)/2)
    res["time_count"] = time_vec
    return res
time_epoch_train = count_time(train)
time_epoch_test = count_time(test) 
#time_epoch

In [171]:
def create_go_up_and_go_down(table):
    up_and_down = pd.DataFrame()
    goes_up = np.zeros(len(table))
    goes_down = np.zeros(len(table))
    for i in range(len(table)):
        if(does_go_up(table.loc[i])):
            goes_up[i] = 1
        if(does_go_down(table.loc[i])):
            goes_down[i] = 1

    up_and_down["goes_up"] = goes_up
    up_and_down["goes_down"] = goes_down
    return up_and_down


def does_go_up(row):
    for i in range(30):
        if(row["velZ_"+str(i)] > 0):
            return True
    return False

def does_go_down(row):
    for i in range(30):
        if(row["velZ_"+str(i)] < 0):
            return True
    return False


#get parabula parameters
def calc_S(row):
    S = []
    for i in range(30):
        if(np.isnan(row["posX_"+str(i)])):
            break
        S.append(((row["posX_"+str(i)])**2 + (row["posY_"+str(i)])**2)**0.5)
    return S

def calc_parabola_params(table):
    res = pd.DataFrame()
    params = np.array([[0 for i in range(len(table))] for j in range(3)])
    for i in range(len(table)):
        row = table.loc[i]
        S = calc_S(row)
        current_params = np.polyfit(S, [row["posZ_"+str(i)] for i in range(len(S))],2)
        cur_f = np.multiply(current_params[0],(np.power(S,2))) + np.multiply(S,current_params[1]) + current_params[2]
        '''
        if(i%1000 == 0):
            plt.plot(S, [row["posZ_"+str(i)] for i in range(len(S))])
            plt.plot(S, cur_f)
            plt.savefig("images/yair_" + str(i) + ".png")
            plt.clf()
            print("iteration " + str(i))
        '''
        params[0][i], params[1][i], params[2][i] = current_params[0], current_params[1], current_params[2]
        res["parabola_parameter_a"] = params[0]
        res["parabola_parameter_b"] = params[1]
        res["parabola_parameter_c"] = params[2]
    return res


In [172]:
#get goes_up and goes_down features:
up_and_down = create_go_up_and_go_down(train)

In [173]:
#get parbola parameters for each features
parabola_params = calc_parabola_params(train)

In [174]:
#create new variable to store the data
new_train =train.copy(deep=True)
new_test =test.copy(deep=True)      

In [175]:
#remove Position columns from data
col_names=list(new_train)
for name in col_names:
    if str(name)[:3] == "pos":
        new_train=new_train.drop(name, 1)

col_names=list(new_test)
for name in col_names:
    if  str(name)[:3] == "pos":
        new_test=new_test.drop(name, 1)  

In [181]:
#create a vector of all the matrices and array which we which to add to the data

df_add_vec_train=[new_train,vel_mag_train,
                  vel_mean_train,acc_df_train,
                  acc_mean_train,angle_df_train,
                  angle_mean_train,angle_change_df_train,
                  angle_change_mean_train,
                  time_epoch_train, up_and_down, parabola_params]

df_add_vec_test=[new_test,vel_mag_test,
                  vel_mean_test,acc_df_test,
                  acc_mean_test,angle_df_test,
                  angle_mean_test,angle_change_df_test,
                  angle_change_mean_test,
                  time_epoch_test]

#merge them to the data
final_train = pd.concat(df_add_vec_train, axis=1)
final_test = pd.concat(df_add_vec_test, axis=1)


In [182]:
pd.set_option('display.max_columns', 500)
final_train.head(10)

,velX_0,velY_0,velZ_0,velX_1,velY_1,velZ_1,velX_2,velY_2,velZ_2,velX_3,velY_3,velZ_3,velX_4,velY_4,velZ_4,velX_5,velY_5,velZ_5,velX_6,velY_6,velZ_6,velX_7,velY_7,velZ_7,velX_8,velY_8,velZ_8,velX_9,velY_9,velZ_9,velX_10,velY_10,velZ_10,velX_11,velY_11,velZ_11,velX_12,velY_12,velZ_12,velX_13,velY_13,velZ_13,velX_14,velY_14,velZ_14,velX_15,velY_15,velZ_15,velX_16,velY_16,velZ_16,velX_17,velY_17,velZ_17,velX_18,velY_18,velZ_18,velX_19,velY_19,velZ_19,velX_20,velY_20,velZ_20,velX_21,velY_21,velZ_21,velX_22,velY_22,velZ_22,velX_23,velY_23,velZ_23,velX_24,velY_24,velZ_24,velX_25,velY_25,velZ_25,velX_26,velY_26,velZ_26,velX_27,velY_27,velZ_27,velX_28,velY_28,velZ_28,velX_29,velY_29,velZ_29,class,vel_magnitude_0,vel_magnitude_1,vel_magnitude_2,vel_magnitude_3,vel_magnitude_4,vel_magnitude_5,vel_magnitude_6,vel_magnitude_7,vel_magnitude_8,vel_magnitude_9,vel_magnitude_10,vel_magnitude_11,vel_magnitude_12,vel_magnitude_13,vel_magnitude_14,vel_magnitude_15,vel_magnitude_16,vel_magnitude_17,vel_magnitude_18,vel_magnitude_19,vel_magnitude_20,vel_magnitude_21,vel_magnitude_22,vel_magnitude_23,vel_magnitude_24,vel_magnitude_25,vel_magnitude_26,vel_magnitude_27,vel_magnitude_28,vel_magnitude_29,vel_mean,accel_0,accel_1,accel_2,accel_3,accel_4,accel_5,accel_6,accel_7,accel_8,accel_9,accel_10,accel_11,accel_12,accel_13,accel_14,accel_15,accel_16,accel_17,accel_18,accel_19,accel_20,accel_21,accel_22,accel_23,accel_24,accel_25,accel_26,accel_27,accel_28,acc_mean,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,angle_10,angle_11,angle_12,angle_13,angle_14,angle_15,angle_16,angle_17,angle_18,angle_19,angle_20,angle_21,angle_22,angle_23,angle_24,angle_25,angle_26,angle_27,angle_28,angle_mean,angle_change_0,angle_change_1,angle_change_2,angle_change_3,angle_change_4,angle_change_5,angle_change_6,angle_change_7,angle_change_8,angle_change_9,angle_change_10,angle_change_11,angle_change_12,angle_change_13,angle_change_14,angle_change_15,angle_change_16,angle_change_17,angle_change_18,angle_change_19,angle_change_20,angle_change_21,angle_change_22,angle_change_23,angle_change_24,angle_change_25,angle_change_26,angle_change_27,angle_change_mean,time_count,goes_up,goes_down,parabola_parameter_a,parabola_parameter_b,parabola_parameter_c
0,486.926974,3.057090,305.119216,483.225599,-0.534244,305.483552,479.884574,2.569264,295.705799,476.162385,-0.844225,284.568090,472.154455,-0.019689,272.206754,466.567232,-1.497475,268.333897,458.682625,-2.764294,259.327302,450.651136,-0.464813,256.915150,446.202964,5.844478,243.623699,447.777201,-1.991051,240.314104,446.003514,-2.218006,224.761679,437.785557,1.055716,225.442226,434.399012,-0.583478,215.780765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,574.634632,571.688259,563.682291,554.716439,545.001235,538.228859,526.923184,518.740838,508.412971,508.192144,499.441555,492.424111,485.040391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.778993,-2.946373,-8.005968,-8.965852,-9.715203,-6.772376,-11.305675,-8.182346,-10.327866,-0.220828,-8.750589,-7.017443,-7.383721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.466187,32.255967,31.663686,30.531277,29.974932,31.251483,28.591722,29.166489,28.249308,29.784352,26.808919,27.311135,27.727107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.443031,0.592281,1.132409,0.556344,1.276551,2.659762,0.574768,0.917181,1.535044,2.975434,0.502216,0.415972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.194360,6.0,1.0,0.0,0,0,478
1,561.240579,0.939582,16.840572,555.568670,0.692598,3.101450,555.637391,-2.662952,5.754135,550.590398,0.646220,-1.185523,549.844356,-2.747061,-5.957418,553.250171,-2.105015,-12.404433,546.458364,4.847240,-22.146437,542.319011,0.783442,-23.550451,542.898667,0.389672,-26.225190,538.531843,

In [183]:
#keep the clean data in CSV files
final_train.to_csv(os.path.join(path,r'final_train_Rafael.csv'), na_rep='NaN',header=True,index=True)
final_test.to_csv(os.path.join(path,r'final_test_Rafael.csv'), na_rep='NaN',header=True,index=True)

In [19]:
'''
#rearenge the column name to enable modeling
final_train.columns=list(final_train)[:91]+[i for i in range(1,125)]
final_test.columns=list(final_train)[:90]+[i for i in range(1,125)]
'''

In [184]:
#seperatre training data into features and labels
Y =pd.DataFrame(final_train['class'])
X =final_train.drop('class', axis=1)

In [185]:
#split data to training and testinf sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
y_train,y_test=np.ravel(y_train),np.ravel(y_test)

In [ ]:
"""
# in case needed it is posible to add an intex to the data 
ind=list(range(0,28746))
final_train=final_train.assign(Index = ind)
#final_train
"""

In [ ]:
xgb_model = xgb.XGBClassifier()

#brute force scan for all parameters, here are the tricks
#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['multi:softmax'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'seed': [1337]}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y_train,n_folds=5, shuffle=True), 
                   scoring='f1_macro',
                   verbose=10, refit=True)

clf.fit(X_train, y_train)

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('F1 score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [ ]:
# a very simple XGboost model
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
#param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 26
param['min_child_weight']=1
param['gamma']=0
param['subsample']=0.8
param['scale_pos_weight']=1
param['colsample_bytree']=0.7
param['learning_rate'] =0.05
param['n_estimators']=1000
param['seed']=123

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 1000
bst = xgb.train(param, xg_train, num_round, watchlist )
# get prediction
pred = np.int_(bst.predict( xg_test ))

In [ ]:
#calculation
f1 = f1_score(y_test, pred,average='macro')
print("F1: %.2f%%" % (f1 * 100.0))

In [ ]:
#A simple Gradient boosting model
train_gbdt=final_train.replace([np.inf],[np.nan])
train_gbdt.fillna(0,inplace=True)
y_gbdt=pd.DataFrame(train_gbdt['class'])
X_gbdt=train_gbdt.drop('class', axis=1)

ss = ShuffleSplit(n_splits=5,random_state=123345,test_size=0.2)
for train_index, test_index in ss.split(X_gbdt,np.ravel(y_gbdt)): 
    X_train , X_test = X_gbdt.loc[train_index,:] , X_gbdt.loc[test_index,:]
    y_train , y_test = y_gbdt.loc[train_index] , y_gbdt.loc[test_index]

gbdt = GradientBoostingClassifier(max_depth=5,subsample=0.8,n_estimators=30)
gbdt.fit(X_train,y_train)
pred = gbdt.predict(X_test)
print (classification_report(pred,y_test))

In [ ]:
#pred

In [ ]:
#y_test

In [ ]:
submission_results = model.predict_proba(test)[:,1]

In [ ]:
sub.to_csv(os.path.join(path,r'Rafael_submission1.csv'),header=True,index=True, index_label='id')